connect duckdb

In [16]:
import duckdb

conn = duckdb.connect("sakila.duckdb")

# determine the maximum number of rows DuckDB will..
# sample from a Pandas DataFrame when automatically analyzing its schema
conn.sql("SET pandas_analyze_sample=100000;")

a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [ ]:
longer_than_3_hour = conn.sql("""

select 
    title, 
    length 
from staging.film
where length > 180
        """).df()

longer_than_3_hour['title'] = longer_than_3_hour['title'].str.title()

# lägger till en header/rubrik med style.setcaption
longer_than_3_hour.style.set_caption("Movies longer than 3 hours").set_table_styles([
    
    {
        "selector": "caption", # sätter att jag vill styla caption (som är rubrik) 
        "props": [ # detta är css-reglerna för min styling - en lista med tuples 
            
            ("font-family", "Arial"),
            ("font-size", "Arial"),
            ("font-size", "20px"),
            ("font-weight", "bold"),
            ("margin-bottom", "5px")
        ] 
    }
])


b) Which movies have the word "love" in its title?
 
Show the following columns, title ,rating length, description

In [ ]:
conn.sql(""" --sql

select
    title,
    rating,
    length, 
    description
    
    from staging.film
    where regexp_matches(title, 'love')
    
;
""").df()

c) Calculate descriptive statistics on the length column, 

The Manager wants, shortest, average, median and
longest movie length

In [ ]:
conn.sql(""" 
         --sql
         select
            min(length) as shortest_movie_min,
            avg(length) as average_length_of_all_movies_min,
            median(length) as median_length_of_all_movies_min,
            max(length) as longest_movie_min
             
            
         from staging.film
         """).df()

d) The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie.

 The Manager wants to know the 10 most expensive movies to rent per day.


https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

In [ ]:

ten_exp_movie = conn.sql(""" --sql
         
    select
        title,
        rental_rate/rental_duration as cost_per_day_usd
            
    from staging.film
    order by cost_per_day_usd desc
    limit 10

        """).df()

# snyggar upp titlarna med funktionen .str.title() och sparar tillbaka ändringen till kolumnen

ten_exp_movie['title'] = ten_exp_movie['title'].str.title()

# lägger till en titel med style.setcaption
ten_exp_movie.style.set_caption("10 most expensive movies").set_table_styles([
    
    {
     "selector": "caption", ## sätter vad jag vill styla, i detta fallet caption = rubrik 
     "props": [ # CSS-reglerna 
         ("font-family", "Arial"),
         ("font-size", "20px"),
         ("font-weight", "bold"),
         ("margin-bottom", "5px")
         ]
    }
    ])

e) Which actors have played in most movies?
 
Show the top 10 actors with the number of movies they have
played in

In [ ]:
actor_vs_movies = conn.sql(""" --sql

select 
    first_name, 
    last_name,
    count(*) as amount_of_movies

from staging.film

inner join
    staging.film_actor on film.film_id = film_actor.film_id

inner join 
    staging.actor on actor.actor_id = film_actor.actor_id

group by actor.actor_id, first_name, last_name
order by amount_of_movies desc
limit 10

;
""").df()

actor_vs_movies['first_name'] = actor_vs_movies['first_name'].str.title()
actor_vs_movies['last_name'] = actor_vs_movies['last_name'].str.title()
actor_vs_movies.style.set_caption("Top 10 actors").set_table_styles(
    
    [
    
    {
        "selector": "caption",
        "props": [
            ("font-family", "Arial"),
            ("font-size", "20px"),
            ("font-weight", "bold"),
            ("margin-bottom", "5px")
        ]
    }
    ]
)


f) Now it's time for you to choose your own question

 to explore the sakila database! Write down 3-5
 
 questions you want to answer and then answer them using pandas and duckdb.



f1) which 10 movies have the highest rental_rate?

(Show title and rental_rate)

In [ ]:
highest_rental = conn.sql("""
--sql
    select title, rental_rate
    from staging.film 
    order by rental_rate desc
    limit 10
;
""").df()

highest_rental['title'] = highest_rental['title'].str.title() 
highest_rental.style.set_caption("Movies with highest rental rate").set_table_styles(
                
    [
        
    {
        "selector": "caption",
        "props": [
            ("font-family", "Arial"),
            ("font-size", "20px"),
            ("font-weight", "bold"),
            ("margin-bottom", "5px")
        ]
            
    }
        
    ]
)

f2) How many movies are in each category?

(Show category name and number of movies)

In [ ]:
movies_per_category = conn.sql("""
--sql
    select name as 'category name', count(*) as "number of movies"
    
    from staging.film
    
    inner join
        staging.film_category on film.film_id = film_category.film_id
    
    inner join  
        staging.category on film_category.category_id = category.category_id 
        
    group by category.name
    
    order by "number of movies" desc
;
""").df()

movies_per_category.style.set_caption("Movies per category").set_table_styles(
    
    [
        
        {
            "selector": "caption",
            "props": [
                ("font-family", "Arial"),
                ("font-size", "20px"),
                ("font-weight", "bold"),
                ("margin-bottom", "5px")
            ]
        }
    
])



f3) Which customers have made the most rentals?

(Show customer name and rental count)

In [ ]:
most_active_renters = conn.sql("""
--sql
    select first_name, count(*) as amount_of_rental
    
    from staging.customer
    
    inner join 
        staging.payment on customer.customer_id = payment.customer_id
    
    inner join 
        staging.rental on payment.rental_id = rental.rental_id
        
    group by customer.customer_id, first_name
    
    order by amount_of_rental desc
    
    limit 10
;
""").df()

most_active_renters['first_name'] = most_active_renters['first_name'].str.title()

most_active_renters.style.set_caption("Most active renters").set_table_styles(
    
    [
        
        {
            "selector": "caption",
            "props": [
                ("font-family", "Arial"),
                ("font-size", "20px"),
                ("font-weight", "bold"),
                ("margin-bottom", "5px")
            ]
        }
    ]
    )

f4) Which movies have the longest rental_duration?

(Show title and rental_duration)

In [ ]:
rental_duration_df = conn.sql("""
--sql
    select title, rental_duration
    
    from staging.film
    order by rental_duration desc
    
    limit 10
;
""").df()

rental_duration_df['title'] = rental_duration_df['title'].str.title()

rental_duration_df.style.set_caption("Top Movies by Rental Duration").set_table_styles(
    [
        {
            "selector": "caption",
            "props": [
                
                ("font-family", "Arial"),
                ("font-size", "20px"),
                ("font-weight", "bold"),
                ("margin-bottom", "5px") 
            ]
        }
    ])

f5) Which cities have the most customers?

show city and amount of customers

In [ ]:
customers_by_city = conn.sql(""" --sql
               
select city, count(*) as amount_of_customer

from staging.customer

inner join
    staging.address on customer.address_id = address.address_id
    
inner join 
    staging.city on address.city_id = city.city_id
    
group by city 

order by amount_of_customer desc
limit 20
;
""").df()

customers_by_city.style.set_caption("Cities Ranked by Number of Customers").set_table_styles(
    [
        {
            
            "selector": "caption",
            "props": [
                
                ("font-family", "Arial"),
                ("font-size", "20px"),
                ("font-weight", "bold"),
                ("margin-bottom", "5px")
            ]
            
        }
    
])


### task 2 - graphs

##### a) Who are our top 5 customers by total spend? 
##### The Manager wants to know so that they can reward them with special offers.
##### Create a bar chart showing the top 5 customers by total spend.

b) How much money does each film category bring in?

Make a bar chart showing total revenue per film category.